# Transcription Factor DNA binding assay data analysis pipeline 

### Version: SGE version (2/28/2018)

##### Niu Du (dniu [at] jcvi.org)
##### J. Craig Venter Institute (JCVI)
##### La Jolla, CA USA

#### require installation of following tools
* fastqc
* multiqc
* GEM - http://groups.csail.mit.edu/cgs/gem/
* MACS2 - https://github.com/taoliu/MACS
* bwa
* Samtools
* Deeptools - https://deeptools.readthedocs.io/en/develop/
* meme suite - http://meme-suite.org/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
import re, os
from tqdm import tqdm
import itertools
from matplotlib_venn import venn3
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,AnnotationBbox)
from matplotlib.cbook import get_sample_data
import matplotlib.image as mpimg

#import customed function
from Dapseq_functions import Tandem_filter

%matplotlib inline

In [ ]:
#0 Init varibles - edit here
os.environ['file_path'] = './Data/DAP_sequence_2016+2018'
os.environ['control'] = 'Halovector_S35_L002_R1_001'
os.environ['project_code'] = '0746'

# Number of threads to use if possible
THREADS = 8 
os.environ['THREADS'] = str(THREADS)

os.environ['gempath'] = '/home/dniu/tools/gem' #java support required 


In [ ]:
#0.1 Create index for reference genome - Only need to do this once if not done yet
!qsub -P $project_code -N index_genome -cwd bwa index -a bwtsw Genome_REF/PT_Chr_only.fa

In [ ]:
#0.2 Create index for reference genome and mask transposon regions
'''This step may not be needed'''
!/home/dniu/tools/convert2bed/convert2bed -i gff -o bed <MASK-REFERENCE.gff> MASK-REFERENCE.bed
!bedtools maskfasta -fi YOUR-REFERENCE-GENOME.fa -bed MASK-REFERENCE.bed -fo MASK-REFERENCE.fa
## Also need to modify genome reference path in step 7.2 and 9 depending on the purpose of use

In [ ]:
#1 Trim raw fastq files 
File_list = os.listdir(os.environ['file_path']+'/')
!mkdir -p $file_path/Trimmed_output
for file in tqdm(File_list):
    if file.endswith(".gz"):
        os.environ['input_name'] = os.path.join(os.environ['file_path'], file)
        os.environ['output_name'] = os.path.join(os.environ['file_path']+"/Trimmed_output", file)
        !qsub -P $project_code -N trim - cwd -e gridout/err -o gridout/out java -jar /home/jiyindna/Desktop/Dubiosys/Genetools/Trimmomatic-0.36/trimmomatic-0.36.jar  \
        SE -phred33 $input_name $output_name \
        ILLUMINACLIP:TruSeq3-SE:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 

In [ ]:
#1.1 Getting sample names for downstream processes
File_list = os.listdir(os.environ['file_path']+"/Trimmed_output/")
Sample_list = [item.split('.')[0] for item in File_list if item.endswith(".gz")]

In [ ]:
#1.2 In this step (1.2) and the following step (1.3), you can manualy delete some of the samples that you do not wish to analyse
with open(os.environ['file_path']+"/Trimmed_output/filelist.txt","w") as f:
    for item in Sample_list:
        #if item.endswith(".gz"):
        f.write("%s\n" % item)

In [ ]:
#1.3 Edit your file then run this step 
with open(os.environ['file_path']+"/Trimmed_output/filelist.txt","r") as f:
        Sub_sample_list = [item.split('\n')[0] for item in tqdm(f)]

In [ ]:
#2 QC the trimmmed files
!mkdir -p ./$project_ID/Trimmed_output/Fastqc
for item in File_list:
    os.environ['item'] = item
    if item.endswith(".gz"):
        !qsub -P $project_code -N trim -cwd -e gridout/err -o gridout/out /home/dniu/tools/FastQC/fastqc ./$project_ID/Trimmed_output/$item \
        --outdir=./$project_ID/Trimmed_output/Fastqc/

In [ ]:
# 2.1 combine all the fastqc report and merge into one multiqc report
!multiqc ./$file_path/Trimmed_output/Fastqc 
!zip -r $file_path.zip multiqc_report.html ./multiqc_data
!rm -r ./multiqc_data
!rm multiqc_report.html

In [ ]:
#3 Seq alignment and write sam files
# https://arxiv.org/abs/1303.3997

#Important parameters for tuning
# -k Minimum seed length. Matches shorter than INT will be missed.
# -B Mismatch penalty. The sequence error rate is approximately: {.75 * exp[-log(4) * B/A]}
# -O Gap open penalty.[6]

!mkdir -p $file_path/Sam_File
for item in Sub_sample_list:
    os.environ['sample_ID'] = item   
    # Note bwa mem does not take masked genome
    !qsub -P $project_code -N BWA.$sample_ID -cwd -pe threaded $THREADS -e gridout/err -o gridout/out \
    ./run_bwa_mem.py -t $THREADS -k 60 -B 7 -O 6 -v 0 -r $path/Genome_REF/PT_Chr_only.fa \
    -i $file_path/Trimmed_output/$sample_ID.fastq.gz -o $file_path/Sam_File/$sample_ID.sam

In [ ]:
#4 filter out unmapped reads and sort mapped reads
!mkdir -p $file_path/Sam_sorted
for item in Sub_sample_list:
    os.environ['sample_ID'] = item
    !qsub -P $project_code -N SORT.$sample_ID -hold_jid BWA.$sample_ID -cwd -pe threaded $THREADS -e gridout/err -o gridout/out \
    ./run_Sam_view_sort.py -t $THREADS -i $file_path/Sam_File/$sample_ID.sam -o $file_path/Sam_sorted/$sample_ID._temp

In [ ]:
# Continue 4 - run after the previous step is finished
for item in tqdm(Sub_sample_list):
    os.environ['sample_ID'] = item
    !samtools rmdup -s $file_path/Sam_sorted/$sample_ID._temp.bam $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam 
    !samtools index $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam 

In [ ]:
#4.1 Vis singals using all mapped reads - Visulization step
!mkdir -p $file_path/bigWig
for item in Sub_sample_list:
    os.environ['sample_ID'] = item
    !qsub -P $project_code -N bamCoverage -cwd -e gridout/err -o gridout/out \
    bamCoverage -b $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam \
    --normalizeUsing BPM \
    --extendReads 300 \
    -o $file_path/bigWig/$sample_ID.bw 


In [ ]:
#4.2 Compare bam files and target peak regions
for item in Sub_sample_list:
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N bamCompare -cwd -e gridout/err -o gridout/out \
        bamCompare  -b1 $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam \
        -b2 $file_path/Sam_sorted/$control._mapped_sorted.bam \
        -o $file_path/bigWig/$sample_ID.peaks.bw \
        --binSize 80 --operation ratio --scaleFactorsMethod SES -n 1000

In [ ]:
#5 MACS2 for peak calling
#-B store all details
# -q/--qvalue The qvalue (minimum FDR) cutoff to call significant regions.
'''Default is 0.05. For broad marks, you can try 0.01 as cutoff. 
Q-values are calculated from p-values using Benjamini-Hochberg procedure.'''

for item in tqdm(Sub_sample_list):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !/u01/dniu/anaconda3/bin/macs2 callpeak -t $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam \
        -c $file_path/Sam_sorted/$control._mapped_sorted.bam \
        -f BAM --outdir $file_path/MACS -g 1.6e7 -n $sample_ID -B -q 0.01 -m 2 50 --verbose=0

In [ ]:
#6 GEM tool for peak calling and motif model creation - use BAM file
# force q value to be 0.001, q = 3, 0.01, q =2
# Only to use GPS data here

# The p-value of the motif occurence. 
'''The p-value is the probability of a random sequence of the same 
length as the motif matching that position of the sequence with a 
score at least as good.'''

# The q-vlavlue of the motif occurence. 
'''The q-value is the estimated false discovery rate if the occurrence 
is accepted as significant.''' 

#The width (bp) to smooth the read distribution. 
'''If it is set to -1, there will be no smoothing (default=30). 
For ChIP-exo, use a smaller number (say 3 or 5) to achieve higher spatial accuracy.'''

# Fold (IP/Control) cutoff to filter predicted events (default=3)
# Depending on the sequencing depth, the sample read

# Proc. Natl Acad. Sci. USA (2003) 100:9440–9445
!rm -r $file_path/GEM_BED
!mkdir $file_path/GEM_BED
for item in Sub_sample_list:
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N GEM.$sample_ID -cwd -pe threaded $THREADS -e gridout/err -o gridout/out \
        java -jar -Xmx16G $gempath/gem.jar --t $THREADS \
        --f BAM \
        --d $gempath/Read_Distribution_default.txt \
        --g ./Genome_REF/PT_genome_size.txt \
        --genome ./Genome_REF/PT_Chr \
        --expt $file_path/Sam_sorted/$sample_ID._mapped_sorted.bam \
        --ctrl $file_path/Sam_sorted/$control._mapped_sorted.bam \
        --outBED \
        --out $file_path/GEM_BED/$sample_ID \
        --k_min 6 --kmax 20 --k_seqs 600 --k_neg_dinu_shuffle


In [ ]:
########## Choose one of the selected option

In [ ]:
#7 Combining peak regions from MACS2 and GEM

# Set lower limit for MACS2 output
min_score = 1
window_size = 80
!mkdir -p $file_path/Combined_BED
for item in tqdm(Sub_sample_list):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        # Read in bed files from MACS2 output
        try:
            df_bed = pd.read_table(os.environ['file_path']+'/MACS/'+item+'_summits.bed',header = None)
            chrs = list(df_bed[df_bed[4]>min_score][0])
            peaks = list(df_bed[df_bed[4]>min_score][1])
            names = ['MACS_'+str(ix) for ix in range(len(df_bed[df_bed[4]>min_score]))]
            scores = list(df_bed[df_bed[4]>min_score][4])

        # Read in location files from GEM output and add to the list

            df_GEM = pd.read_table(os.environ['file_path']+'/GEM_BED/'+item+'/'+item+'.GEM_events.txt')
            chrs = chrs + ['chr'+pos.split(':')[0] for pos in df_GEM[df_GEM['Fold']>min_score].Position]
            peaks = peaks + [int(pos.split(':')[1]) for pos in df_GEM[df_GEM['Fold']>min_score].Position]
            names = names + ['GEM_'+str(ix) for ix in range(len(df_GEM[df_GEM['Fold']>min_score]))]
            scores = scores + list(df_GEM[df_GEM['Fold']>min_score].Fold)   
            
            with open(os.environ['file_path']+'/Combined_BED/'+item+'.combined.bed','w') as handle:
                for c,p,n,s in zip(chrs,peaks,names,scores):
                    handle.write(c+'\t'+str(p-int(window_size/2))+'\t'+str(p+int(window_size/2))+'\t'+n+'\t'+str(s)+'\n')

        except:
            continue

In [ ]:
#7.1 Generate bed files based on the output from GEM and MACS2, and compare the intercets 
min_score = 1
window_size = 80

# Get all valid files from previous step
Sub_sample_list1 = [name.split('.')[0] for name in os.listdir(os.environ['file_path']+"/Combined_BED/") if name.endswith('bed')]

!mkdir -p $file_path/compare_bed
for item in tqdm(Sub_sample_list1):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        
        df_bed = pd.read_table(os.environ['file_path']+'/MACS/'+item+'_summits.bed',header = None)
        chrs = list(df_bed[df_bed[4]>min_score][0])
        peaks = list(df_bed[df_bed[4]>min_score][1])
        names = ['MACS_'+str(ix) for ix in range(len(df_bed[df_bed[4]>min_score]))]
        scores = list(df_bed[df_bed[4]>min_score][4])
        
        with open(os.environ['file_path']+'/compare_bed/'+item+'.MACS.bed','w') as handle:
            for c,p,n,s in zip(chrs,peaks,names,scores):
                handle.write(c+'\t'+str(p-int(window_size/2))+'\t'+str(p+int(window_size/2))+'\t'+n+'\t'+str(s)+'\n')
        
        df_GEM = pd.read_table(os.environ['file_path']+'/GEM_BED/'+item+'/'+item+'.GEM_events.txt')
        chrs = ['chr'+pos.split(':')[0] for pos in df_GEM[df_GEM['Fold']>min_score].Position]
        peaks = [int(pos.split(':')[1]) for pos in df_GEM[df_GEM['Fold']>min_score].Position]
        names = ['GEM_'+str(ix) for ix in range(len(df_GEM[df_GEM['Fold']>min_score]))]
        scores = list(df_GEM[df_GEM['Fold']>min_score].Fold)
        
        with open(os.environ['file_path']+'/compare_bed/'+item+'.GEM.bed','w') as handle:
            for c,p,n,s in zip(chrs,peaks,names,scores):
                handle.write(c+'\t'+str(p-int(window_size/2))+'\t'+str(p+int(window_size/2))+'\t'+n+'\t'+str(s)+'\n')
    
        
        !bedtools intersect -wo -a $file_path/compare_bed/$sample_ID.MACS.bed -b $file_path/compare_bed/$sample_ID.GEM.bed > $file_path/compare_bed/$sample_ID.compare.bed 

In [ ]:
#7.2 Generate fasta files from bed files and remove duplicated sequences
!mkdir -p $file_path/fasta
for item in tqdm(Sub_sample_list1):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        os.environ['fasta_file'] = os.environ['file_path']+'/fasta/'+os.environ['sample_ID']+'.fasta'
        !bedtools getfasta -fo $fasta_file -fi ./Genome_REF/PT_Chr_only.fa \
        -bed $file_path/Combined_BED/$sample_ID.combined.bed
        
        key_list = []
        dict_fasta = {}
        with open(os.environ['fasta_file'],'r') as handle:
            with open(os.environ['fasta_file']+'.nodup','w') as output_handle:
                for rec in SeqIO.parse(handle,'fasta'):
                    if rec.name not in key_list:
                        key_list.append(rec.name)
                        output_handle.write('>'+rec.name+'\n'+str(rec.seq)+'\n')

In [ ]:
#7.3 Filter out tandem sequences 

# P of 6-mer show up in n base pair
# m time n/(4^6)^m - 1 time 0.12 -2 times 0.3e-3 -3 times 7.3e-9
for count,item in enumerate(Sub_sample_list1):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        print(str(count+1)+'/'+str(len(Sub_sample_list1)),item)
        fasta_file = os.environ['file_path']+'/fasta/'+os.environ['sample_ID']+'.fasta.nodup'
        Tandem_filter(fasta_file,k = 6,k_max = 4)    

In [ ]:
#8 Use meme suite to build motif model

'''-objfun ce - This objective function scores motifs based on their enrichment in the central regions of the primary sequences, which must all be of same the length. (See option -cefrac, below.) 
The width and number of sites of each motif are determined using the binomial test. '''
# -mod anr/oops/zoops
# -ws The gap extension cost for creating the alignments used to trim the motif.

for item in Sub_sample_list1:
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N MEME.$sample_ID -cwd -e gridout/err -o gridout/out \
        meme $file_path/fasta/$sample_ID.fasta.nodup \
        -nmotifs 1 -minw 4 -maxw 12 -dna -mod oops -nostatus \
        -oc $file_path/GEM_BED/$sample_ID-meme

In [ ]:
#9 Locate motif loci in the genome
for item in Sub_sample_list1:
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N FIMO.$sample_ID -hold_jid MEME.$sample_ID -cwd -e gridout/err -o gridout/out \
        fimo -verbosity 1 --thresh 1e-5\
        -oc $file_path/GEM_BED/$sample_ID-fimo/ \
        $file_path/GEM_BED/$sample_ID-meme/meme.txt \
        ./Genome_REF/PT_Chr_only.fa

In [ ]:
#9.1 Convert gff to bed
Sub_sample_list2 = [name.split('-')[0] for name in os.listdir(os.environ['file_path']+"/GEM_BED/") if name.endswith('fimo')]

for item in Sub_sample_list2:
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N gff2bed.$sample_ID -hold_jid FIMO.$sample_ID -cwd -e gridout/err \
        -o $file_path/GEM_BED/$sample_ID-fimo/fimo.bed \
        gff2bed -i $file_path/GEM_BED/$sample_ID-fimo/fimo.gff

In [ ]:
#9.2 Compare motif to peaks called by GEM and MACS2

def get_type(x):
    if len(x.split(','))>1:
        return 0
    elif 'MACS' in x:
        return 1
    elif 'GEM' in x:
        return 2
    else:
        return -1
    
venn_set = {}
for item in tqdm(Sub_sample_list2):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        try:
            !bedtools intersect -wa -a $file_path/GEM_BED/$sample_ID-fimo/fimo.bed -b $file_path/compare_bed/$sample_ID.MACS.bed \
            > $file_path/compare_bed/$sample_ID.MACS_peak.bed
            !bedtools intersect -wa -a $file_path/GEM_BED/$sample_ID-fimo/fimo.bed -b $file_path/compare_bed/$sample_ID.GEM.bed \
            > $file_path/compare_bed/$sample_ID.GEM_peak.bed
            !bedtools intersect -wa -a $file_path/compare_bed/$sample_ID.MACS.bed -b $file_path/compare_bed/$sample_ID.GEM.bed \
            > $file_path/compare_bed/$sample_ID.compare_peak.bed
            # !bedtools intersect -wa -a $file_path/GEM_BED/$sample_ID-fimo/fimo.bed -b $file_path/compare_bed/$sample_ID.MACS.bed $file_path/compare_bed/$sample_ID.GEM.bed \
            # > $file_path/compare_bed/$sample_ID.all_peak.bed

            df_MACS = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.MACS.bed',header = None)
            df_GEM = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.GEM.bed',header = None)
            df_Motif = pd.read_table(os.environ['file_path']+'/GEM_BED/'+os.environ['sample_ID']+'-fimo/fimo.bed',header = None) 

            df_compare_MACS_peak = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.MACS_peak.bed',header = None)
            #df_compare_MACS_peak = df_compare_MACS_peak.drop_duplicates(7)
            df_compare_GEM_peak = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.GEM_peak.bed',header = None)
            #df_compare_GEM_peak = df_compare_GEM_peak.drop_duplicates(7)
            df_compare = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.compare_peak.bed',header = None)

            # compare all three 
            df_compare_motif = pd.read_table(os.environ['file_path']+'/compare_bed/'+os.environ['sample_ID']+'.all_peak.bed',header = None)
            df_compare_motif = df_compare_motif.drop_duplicates()
            
            All_overlay = len(set(df_compare_MACS_peak[3]).intersection(df_compare_GEM_peak[3]))
            
            # Unique to MACS
            MACS_only = len(df_MACS)-len(df_compare)-len(df_compare_MACS_peak)+All_overlay
            # Unique to GEM
            GEM_only = len(df_GEM)-len(df_compare)-len(df_compare_GEM_peak)+All_overlay
            # Unique to MOTIF
            MOTIF_only = len(df_Motif) - len(df_compare_MACS_peak) - len(df_compare_GEM_peak) + All_overlay

            # GEM overlap MACS only
            GEM_MACS_only = len(df_compare) - All_overlay
            # MACS overlap MOTIF only
            MACS_MOTIF_only = len(df_compare_MACS_peak) - All_overlay
            # GEM overlap MOTIF only
            GEM_MOTIF_only = len(df_compare_GEM_peak) - All_overlay
            
            # Get the overaly motif and write into bed format
            df_Motif.set_index(3).loc[set(df_compare_MACS_peak[3]).union(df_compare_GEM_peak[3])].reset_index()[[0,1,2,3]].to_csv(os.environ['file_path']+'/Combined_BED/'+os.environ['sample_ID']+'.intersection.bed',header = None, index = False, sep = '\t')
            venn_set[item] = [MACS_only, GEM_only, GEM_MACS_only, MOTIF_only,MACS_MOTIF_only,GEM_MOTIF_only,All_overlay]
        except:
            continue

In [ ]:
#10 Visulize the final results
%matplotlib inline
f,ax = plt.subplots(2*int(len(venn_set)/4)+2,4,figsize = (24,max(8,len(venn_set)*2)))
ax = ax.ravel()
[axi.set_axis_off() for axi in ax.ravel()]  
for ix,sets in enumerate(venn_set):
    ax[2*ix+1].set_title(str(sets))
    venn3(subsets = (venn_set[sets]),set_labels = ('MACS', 'GEM', 'Motif'),ax = ax[2*ix+1],)
    ax[2*ix].set_title(str(sets))
    img = mpimg.imread(os.environ['file_path']+'/GEM_BED/'+sets+'-meme/logo1.png')
    ax[2*ix].imshow(img);



In [ ]:
#11 Find motif that its locaiton overlaps with GEM and MACS targeted peaks

!mkdir -p $file_path/fasta
for item in tqdm(list(venn_set.keys())):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        os.environ['fasta_file'] = os.environ['file_path']+'/fasta/'+os.environ['sample_ID']+'.motif.fasta'
        !bedtools getfasta -fo $fasta_file -fi ./Genome_REF/PT_Chr_only.fa \
        -bed $file_path/Combined_BED/$sample_ID.intersection.bed

In [ ]:
#12 Do another round of MEME to build better motif model
for item in list(venn_set.keys()):
    if os.environ['control'] not in item:
        os.environ['sample_ID'] = item
        !qsub -P $project_code -N MEME2.$sample_ID -cwd -e gridout/err -o gridout/out \
        meme $file_path/fasta/$sample_ID.motif.fasta \
        -nmotifs 1 -minw 4 -maxw 12 -dna -mod oops -nostatus \
        -oc $file_path/GEM_BED/$sample_ID-meme-intersection


In [ ]:
f,ax = plt.subplots(2*int(len(venn_set)/4)+2,4,figsize = (24,max(8,len(venn_set)*2)))
ax = ax.ravel()
[axi.set_axis_off() for axi in ax.ravel()]  
for ix,sets in enumerate(venn_set):
    ax[2*ix+1].set_title(str(sets))
    venn3(subsets = (venn_set[sets]),set_labels = ('MACS', 'GEM', 'Motif'),ax = ax[2*ix+1],)
    ax[2*ix].set_title(str(sets))
    img = mpimg.imread(os.environ['file_path']+'/GEM_BED/'+sets+'-meme-intersection/logo1.png')
    ax[2*ix].imshow(img);

In [ ]:
#10 Save all useful output files

!rm big_Wig.zip
!zip -r -q -J -D big_Wig.zip $file_path/GEM_BED/*-fimo/fimo.gff $file_path/bigWig/

In [ ]:
!rm motif_locaiton.zip
!zip -q -r -D -J motif_locaiton.zip $file_path/Combined_BED/*.intersection.bed